# Pretrained DinoV2 model

In [1]:
%load_ext autoreload
%autoreload 2

Make directory to store the models

In [ ]:
# ! mkdir -p /mnt/data/models
# %cd /mnt/data/models

copy the file from GCS to `/mnt/data/models` directory

In [ ]:
# ! gsutil cp gs://dsgt-clef-plantclef-2024/data/models/PlantNet_PlantCLEF2024_pretrained_models_on_the_flora_of_south-western_europe.tar /mnt/data/models/

Unzip the `.tar` file

In [ ]:
# ! tar -xvf /mnt/data/models/PlantNet_PlantCLEF2024_pretrained_models_on_the_flora_of_south-western_europe.tar -C /mnt/data/models

use pretrained model for inference

In [ ]:
import pandas as pd
from urllib.request import urlopen
from PIL import Image
import timm
import torch


def load_class_mapping(class_list_file):
    with open(class_list_file) as f:
        class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
    return class_index_to_class_name


def load_species_mapping(species_map_file):
    df = pd.read_csv(species_map_file, sep=";", quoting=1, dtype={"species_id": str})
    df = df.set_index("species_id")
    return df["species"].to_dict()


def main(image_url, class_mapping, species_mapping, pretrained_path):
    cid_to_spid = load_class_mapping(class_mapping)
    spid_to_sp = load_species_mapping(species_mapping)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = timm.create_model(
        "vit_base_patch14_reg4_dinov2.lvd142m",
        pretrained=False,
        num_classes=len(cid_to_spid),
        checkpoint_path=pretrained_path,
    )
    model = model.to(device)
    model = model.eval()

    # get model specific transforms (normalization, resize)
    data_config = timm.data.resolve_model_data_config(model)
    transforms = timm.data.create_transform(**data_config, is_training=False)

    img = None
    if "https://" in image_url or "http://" in image_url:
        img = Image.open(urlopen(image_url))
    elif image_url != None:
        img = Image.open(image_url)

    if img != None:
        img = transforms(img).unsqueeze(0)
        img = img.to(device)
        output = model(img)  # unsqueeze single image into batch of 1
        top5_probabilities, top5_class_indices = torch.topk(
            output.softmax(dim=1) * 100, k=5
        )
        top5_probabilities = top5_probabilities.cpu().detach().numpy()
        top5_class_indices = top5_class_indices.cpu().detach().numpy()

        for proba, cid in zip(top5_probabilities[0], top5_class_indices[0]):
            species_id = cid_to_spid[cid]
            species = spid_to_sp[species_id]
            print(species_id, species, proba)

In [ ]:
# Define your paths and image URL directly
path = "/mnt/data/models/pretrained_models"
image_url = "https://lab.plantnet.org/LifeCLEF/PlantCLEF2024/single_plant_training_data/PlantCLEF2024singleplanttrainingdata/test/1361687/bd2d3830ac3270218ba82fd24e2290becd01317c.jpg"
class_mapping_file = f"{path}/class_mapping.txt"
species_mapping_file = f"{path}/species_id_to_name.txt"
model_path = "vit_base_patch14_reg4_dinov2_lvd142m_pc24_onlyclassifier_then_all"
pretrained_path = f"{path}/{model_path}/model_best.pth.tar"

main(image_url, class_mapping_file, species_mapping_file, pretrained_path)

In [2]:
from plantclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 14:26:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/11 14:26:43 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:
# get dataframes
gcs_path = "gs://dsgt-clef-plantclef-2024"
test_data_path = "data/parquet_files/PlantCLEF2024_test"

# paths to dataframe
test_path = f"{gcs_path}/{test_data_path}"
# read data
test_df = spark.read.parquet(test_path)
# show
test_df.show(n=5, truncate=50)

+-------------------------------------------+------------------------+--------------------------------------------------+
|                                       path|              image_name|                                              data|
+-------------------------------------------+------------------------+--------------------------------------------------+
| /PlantCLEF2024test/CBN-Pla-B4-20160728.jpg| CBN-Pla-B4-20160728.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-D3-20130808.jpg| CBN-Pla-D3-20130808.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
|/PlantCLEF2024test/CBN-PdlC-E4-20150701.jpg|CBN-PdlC-E4-20150701.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-F5-20150901.jpg| CBN-Pla-F5-20150901.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
| /PlantCLEF2024test/CBN-Pla-D1-20180724.jpg| CBN-Pla-D1-20180724.jpg|[FF D8 FF E0 00 10 4A 46 49 46 00 01 01 01 00 4...|
+-----------------------

In [4]:
test_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- image_name: string (nullable = true)
 |-- data: binary (nullable = true)



In [35]:
limit_df = test_df.limit(10).cache()
limit_df.count()

10

In [101]:
import io

import numpy as np
import pandas as pd
import timm
import torch
from PIL import Image
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, MapType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from plantclef.model_setup import setup_pretrained_model
from pyspark.sql import DataFrame
from pyspark.ml.functions import vector_to_array, predict_batch_udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer

In [180]:
class PretrainedDinoV2(
    Transformer,
    HasInputCol,
    HasOutputCol,
    DefaultParamsReadable,
    DefaultParamsWritable,
):
    def __init__(
        self,
        pretrained_path: str,
        input_col: str = "input",
        output_col: str = "output",
        model_name: str = "vit_base_patch14_reg4_dinov2.lvd142m",
        batch_size: int = 8,
    ):
        super().__init__()
        self._setDefault(inputCol=input_col, outputCol=output_col)
        self.model_name = model_name
        self.batch_size = batch_size
        self.pretrained_path = pretrained_path
        self.num_classes = 7806  # total number of plant species
        self.local_directory = "/mnt/data/models/pretrained_models"
        self.class_mapping_file = f"{self.local_directory}/class_mapping.txt"
        # Model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = timm.create_model(
            self.model_name,
            pretrained=False,
            num_classes=self.num_classes,
            checkpoint_path=self.pretrained_path,
        )
        self.model.to(self.device)
        self.model.eval()
        # Data transform
        self.data_config = timm.data.resolve_model_data_config(self.model)
        self.transforms = timm.data.create_transform(
            **self.data_config, is_training=False
        )
        self.sql_statement = "SELECT image_name, dino_logits FROM __THIS__"
        self.cid_to_spid = self._load_class_mapping()

    def _load_class_mapping(self):
        with open(self.class_mapping_file) as f:
            class_index_to_class_name = {i: line.strip() for i, line in enumerate(f)}
        return class_index_to_class_name

    def _make_predict_fn(self):
        """Return PredictBatchFunction using a closure over the model"""

        def predict(input_data: list) -> list:
            # Load all inputs into a batch of images
            img = Image.open(io.BytesIO(input_data))
            # Transform and stack images to a single tensor
            processed_image = self.transforms(img).unsqueeze(0).to(self.device)

            with torch.no_grad():
                outputs = self.model(processed_image)
                probabilities = torch.softmax(outputs, dim=1) * 100
                top_probs, top_indices = torch.topk(probabilities, k=20)

            top_probs = top_probs.cpu().numpy()[0]
            top_indices = top_indices.cpu().numpy()[0]

            # Convert top indices and probabilities to a dictionary
            result = [
                {self.cid_to_spid.get(index, "Unknown"): float(prob)}
                for index, prob in zip(top_indices, top_probs)
            ]
            return result

        return predict

    def _transform(self, df: DataFrame):
        # Create a UDF from the predict function
        predict_fn = self._make_predict_fn()
        predict_udf = F.udf(predict_fn, ArrayType(MapType(StringType(), FloatType())))

        return df.withColumn(
            self.getOutputCol(),
            predict_udf(self.getInputCol()),
        )

    def transform(self, df) -> DataFrame:
        transformed = self._transform(df)

        for c in self.feature_columns:
            # check if the feature is a vector and convert it to an array
            if "array" in transformed.schema[c].simpleString():
                continue
            transformed = transformed.withColumn(c, vector_to_array(F.col(c)))
        return transformed

    @property
    def feature_columns(self) -> list:
        return ["dino_logits"]

    def pipeline(self):
        return Pipeline(stages=[self, SQLTransformer(statement=self.sql_statement)])

    def run(self, df: DataFrame) -> DataFrame:
        model = self.pipeline().fit(df)
        transformed = model.transform(df)

        return transformed

In [181]:
pretrained_path = setup_pretrained_model()
pretrained_dino = PretrainedDinoV2(
    pretrained_path=pretrained_path,
    input_col="data",
    output_col="dino_logits",
)

Model already exists. Skipping download and extraction.


In [183]:
transformed_df = pretrained_dino.run(df=limit_df)
transformed_df.show(n=5, truncate=50)

results: [{'1361281': 7.845359802246094}, {'1741880': 4.131918430328369}, {'1741903': 2.7131903171539307}, {'1397070': 2.321333169937134}, {'1362271': 2.125537157058716}, {'1388766': 1.5371482372283936}, {'1395807': 1.430025577545166}, {'1363407': 1.3783882856369019}, {'1396640': 1.1620782613754272}, {'1358379': 0.8249717950820923}, {'1544716': 0.7349836826324463}, {'1391317': 0.6994383931159973}, {'1390929': 0.6965225338935852}, {'1390899': 0.6008772850036621}, {'1519650': 0.5890170335769653}, {'1394999': 0.515813410282135}, {'1396869': 0.48324480652809143}, {'1389059': 0.48096683621406555}, {'1395868': 0.44235530495643616}, {'1395862': 0.4272904694080353}]
results: [{'1741706': 6.709542274475098}, {'1741903': 5.509812831878662}, {'1741880': 3.1866233348846436}, {'1737559': 2.670597553253174}, {'1395862': 2.496324300765991}, {'1397524': 2.423849582672119}, {'1361281': 2.0336947441101074}, {'1397070': 1.7238483428955078}, {'1395807': 1.6485413312911987}, {'1363266': 1.6216201782226562}

+------------------------+--------------------------------------------------+
|              image_name|                                       dino_logits|
+------------------------+--------------------------------------------------+
| CBN-Pla-B4-20160728.jpg|[{1361281 -> 7.84536}, {1741880 -> 4.1319184}, ...|
| CBN-Pla-D3-20130808.jpg|[{1741706 -> 6.7095423}, {1741903 -> 5.509813},...|
|CBN-PdlC-E4-20150701.jpg|[{1392323 -> 6.9817915}, {1361281 -> 5.2335153}...|
| CBN-Pla-F5-20150901.jpg|[{1361281 -> 14.984538}, {1390764 -> 10.862804}...|
| CBN-Pla-D1-20180724.jpg|[{1361281 -> 35.631886}, {1651363 -> 21.19073},...|
+------------------------+--------------------------------------------------+
only showing top 5 rows



results: [{'1361281': 7.391123294830322}, {'1743474': 2.836224317550659}, {'1741880': 2.1084747314453125}, {'1394311': 1.8508533239364624}, {'1397463': 1.6546798944473267}, {'1414270': 1.255638599395752}, {'1478563': 0.6917396187782288}, {'1359282': 0.5534758567810059}, {'1398779': 0.5463439226150513}, {'1369068': 0.5071946978569031}, {'1390929': 0.45880624651908875}, {'1396717': 0.45795753598213196}, {'1395807': 0.45595788955688477}, {'1412857': 0.45178186893463135}, {'1397867': 0.4127252399921417}, {'1411089': 0.38960501551628113}, {'1398772': 0.38724297285079956}, {'1361284': 0.3817717730998993}, {'1397495': 0.38029518723487854}, {'1397535': 0.37551528215408325}]
